Before starting the algorithm, we first implement some methods. Let's discuss the methods first.
Some variables as declared below.

In [ ]:
truth_val=[]    # list to store truth assignments
unit_cl=[]      # list to store temporarily the unit clause
no_of_var=0     # Number of Variables
no_of_clause=0  # Number of Clause

The following function is just to read input file and store. 

In [ ]:
# method to read input
def read_file():
  ip_file = input("Please Enter The File Name: ")
  global no_of_var
  global no_of_clause
  formula=[]
  clause=[]
 
  with open(ip_file) as ipf:
    for line in ipf:
      if line[0]=='c':
        continue
      elif line[0]=='p':
        a,b,c,d = line.split()
        no_of_var = int(c)
        no_of_clause = int(d)
      else:
        clause = list(map(int,line.split()))
        clause.pop()
        formula.append(clause)
  return formula

We need to initialize the truth assignment first. Through out the programme if a variable is yet to assign a truth value, -1 is assigned to it. Further for truth value "True" 1 and for "False" 0 is assigned.

In [ ]:
# methode truth value initialisation 
def initialisation():
  global truth_val
  global no_of_var
  for i in range(0,no_of_var):
    truth_val.append(-1)

The following method writes output in a text file "*output.txt*". The first line says about satisfiability. If satisfiable, an satisfying assignment is follows. There maybe more satisfying assignments, but only one is shown.
 In this function an argument namely "`satisfiable`" is taken. The DPLL algorithm is implemented in such way that it returns **True** if *satisfiable* else returns **False**. This returned value is passed as an argument in this function. If **True**, it also checks the list namely `truth_val`, where we store the truth assignment, and write the truth assignments accordingly. If some variables can be assigned either of the truth value, these are kept as **False** by default.


In [ ]:
# method to print output
def print_output(satisfiable):
  global truth_val
  global no_of_var
  opf = open("output.txt",'w')
  if satisfiable == False:
    opf.write("The Given Set of Clauses is not Satisfiable")
  else:
    opf.write("The Given Set of Clauses is Satisfiable with truth assignment as follows:")
    for i in range(0,no_of_var):
      opf.write("\n" + str(i+1) + " : ")
      if truth_val[i]==1:
        opf.write("True")
      else:
        opf.write("False")


The following function stores the truth assignment of a literal to its corresponding location in the list `truth_val`. We need to check whether the variable is already assigned a truth value. If the complemented Truth value is already assigned, it is a contradiction and we return `False`. Otherwise repective truth value is assigned to it.


In [ ]:
# method to assign truth value
def assign_truth_value(literal):
  global truth_val
  absolute_val = abs(literal)
  if truth_val[absolute_val-1] == -1: # checking whether the variable is already assigned a truth value
    if literal > 0:
      truth_val[absolute_val-1] = 1
    else:
      truth_val[absolute_val-1] = 0
    return True
  return False

The following function search clauses with length 1. As soon an it finds one, it calls the above function `assign_truth_value()`. If assigned, the literal is stored in the unit clause list namely `unit_cl` and the clause is removed from the list `formula`

In [ ]:
# methode to find unit clause and remove
def find_unit_clause(formula):
  global truth_val
  global unit_cl
  ucl_found = False
  for clause in formula:
    if len(clause)==1 and truth_val[abs(clause[0])-1]==-1 : # seraching for clause with length 1
      ucl_found = True
      unit_cl.append(clause[0]) # literal is added to the unit clause list.
      if assign_truth_value(clause[0]):
        formula.remove(clause)
  return ucl_found , formula

When we find an unit clause, we assign **True** to the literal contained in the unit clause. Thus every clause containing that literal is satisfied. Therefore we remove those redundant clauses. In contrast, if the negation of the literal is present in some clause, those are removed as cannot be satisfied with the assignment. 

In [ ]:
# method to update formula : remove clause containing literal : remove negated literal
def uptate_formula(formula, literal):
  neg_literal = -1*literal
  stack = []
  i=0
  for clause in formula:
    flag = False
    for lit in clause:
      if lit == literal: # checking wheter cantains the literal. If yes, set a flag.
        flag = True
    if flag: # if flag is set, we store the index in a stack to remove the clause further.
      stack.append(i) 
    i=i+1
  while stack: # removing the clauses indexed above from last. 
    formula.pop(stack.pop())
  updated_formula = [[lit for lit in clause if lit != neg_literal] for clause in formula]
          #the negation of the literal is avoided. Selecting rest the formula is updated.
  return updated_formula

The following method is implementation of unit resolution.  First we sreach for unit clauses calling the above mentioned method `find_unit_clause()` which returns wheter a unit clause is found along with a list namely `unit_cl` containing the unit clauses. While there is a unit clause in the list, we update the formula as described in the above method `uptate_formula()`. This is done as long as at least a unit clause is found. 

In [ ]:
# methode Unit Resolution
def unit_resolution(formula):
  global unit_cl
  ucl_found = True
  while ucl_found:
    ucl_found, formula = find_unit_clause(formula)
    while unit_cl:
      formula = uptate_formula(formula, unit_cl.pop() )
  return formula

In the contrast to the above method, if there is not any unit clause, a literal is choosed and assigned a truth value to it. Here simply the first literal of the formula is chooed. Performance can be improved with a claver choice.

In [ ]:
# method to choose literal
def choose_literal(formula):
  for clause in formula:
    if len(clause)!=0:
      return clause[0]

The following method appends the chosen literal as an unit clause to the formula. Later when searched for a unit clause, it will be found.

In [ ]:
# method to append the chosen literal as an unit clause to the formula.
def append_formula(formula, literal):
  formula.append([literal])
  return formula

In [ ]:
# method to check whether the formula is empty.
def is_empty(formula):
    for clause in formula:
        if clause:
            return False
    return True

Finally, its time to assemble all the above mentioned method here. The following method is an implementation of the DPLL algorithm. 
First of all it is checked whether the formula is empty (which is a satisfiable case) and whether it contains a empty clause (which is a unsatisfiable case). There after we remove the unit clauses (until there is any) along with the clauses containing it and negation of the literal from the clauses. This is done with the method `unit_resolution()`. 
As the formula is updated, again we check for empty clauses and whether the formula is empty. 
Is it comes out of the unit resolution methode, there won't be any unit clause left. So to proced further a literal has to be choosen. Here we have two cases, one when the choosen literal is assign a truth value **True** and other one when assigned **False**. In both the cases we add the literal as an unit clause to the formula and `DPLL()` is called recursively.
Here we first assign **True** and proceed. If we find a contradiction, we backtrack and assign **False** and proceed. In this process we keep updating the formula and the list containing the truth assignments. So we keep a backup of those as `truth_val_backup` and `formula_backup`.

In [ ]:
# The DPLL Algorithm 
def DPLL(formula):
  if any([len(clause)==0 for clause in formula]): # searching for an empty clause
    return False
  if is_empty(formula): # checking whether the formula is empty
    return True
  formula = unit_resolution(formula)
  if any([len(clause)==0 for clause in formula]): # searching for an empty clause
    return False
  if is_empty(formula): # checking whether the formula is empty
    return True
  p = choose_literal(formula)
  global truth_val
  truth_val_backup = truth_val.copy() # truth assignments backup
  formula_backup = formula.copy() # formula backup
  if DPLL(append_formula(formula, p)): # calling DPLL recursively assigning True to the choosen literal
    return True
  truth_val = truth_val_backup.copy() # reset truth assignments
  formula = formula_backup.copy() # reset formula
  return DPLL(append_formula(formula, -p)) # calling DPLL recursively assigning True to the choosen literal


In [ ]:
if __name__=="__main__":
  formula=[]
  formula = read_file()
  initialisation()
  print_output(DPLL(formula))

Please Enter The File Name: sample.txt
